In [1]:
def _amount_():
    try:
        amount = input()
        if amount not in str(list(range(0,10))):
            raise TypeError('Amount must be numerical')
    except TypeError:
        _amount_()
    return float(amount)

class Account:
    def __init__(self, account_id,balance, interest = 0):
        
        '''checking if account id is string'''

        if not isinstance(account_id,str):
            raise TypeError('Account ID must be string')    
        for id in account_id:
            if id not in str(list(range(0,10))):
                raise ValueError('Account ID must be within 0-9')
        for num in balance:
            if num not in str(list(range(0,10))):
                raise ValueError('Balance must be number')
        
        '''assign the attributes'''

        self.account_id = account_id
        self.balance = float(balance)
        self.interest = interest

    '''print message'''

    def __str__(self):
        return f'{self.account_id},{round(self.balance,2)},{self.interest}'

    def _withdraw_(self):

        '''prompt the user to choose amount to withdraw'''

        print('Amount: ',flush = True)
        deducted_amount = _amount_()

        '''prevent user from withdrawing more than their balance '''

        try:
            self.balance -=  deducted_amount
            if self.balance < 0:
                raise ValueError()
        except ValueError:
            print('Cannot proceed your withdrawal')
            self.balance += deducted_amount
    
class Checking(Account):

    '''inherit account class and force interest to be 0'''
    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 0
    
    '''return message'''

    def __str__(self):
        return f'{self.account_id}  {round(float(self.balance),2)}'

class Savings(Account):

    '''inherit account class and force interest to be 1'''

    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 1
    
    '''return message'''

    def __str__(self):
        return f'{self.account_id} {round(float(self.balance),2)}'
    
## Ivan: for Credit class
def __credit_card_charge__(self):

    '''promt user to choose amount'''

    print('Amount: ', flush = True)
    increased_amount = _amount_()

    '''prevent user to charge more than their limit'''
    
    try:
        self.balance += increased_amount
        if increased_amount > self.limit:
            raise ValueError()
    except ValueError:
        print('You have passed your credit limit')
        self.balance -= increased_amount


In [ ]:
'''for storing newly created Customer object'''
customer_collection = {}

def name_lookup():

    '''prompt user to put in only valid customer name and account type'''

    try:
        account_name = input('Customer name:')
        account_type = input('Checking or Savings or Credit')
        if account_name not in customer_collection.keys() and account_type not in ['Savings','savings','checking','Checking','credit',"Credit"]:
            raise ValueError('invalid name/account type')
    except ValueError:
        name_lookup(account_name)
    return account_name,account_type

def interface():

    ## Ghaith & Dallen: import function HERE

    '''display options and prompt user to choose'''

    print(['1. View Customer\n',
            '2. Deposit\n',
            '3. Withdraw\n',
            '4. Credit Card Charge\n',
            '5. Credit Card Payment\n',
            '6. Exit\n'])
    user_choice = input('Enter your 1/2/3/4/5/6')
    while user_choice not in str(list(range(1,8))):
        user_choice = input('Enter your 1/2/3/4/5/6')

    '''withdraw action'''

    ## Ghaith & Dallen: View customer and deposit HERE 
    ## Change if user_choise == '3  to elif 

    if user_choice == '3':

        '''prompt the user to choose customer name and account type'''
        '''avoid typo/invalid name/account'''
                
        account_name, account_type = name_lookup

        
        '''proceed withdrawing by calling withdraw method for corresponding account type'''

        if account_type in ['Checking','checking']:
            customer_collection[account_name].checking._withdraw_()
        elif account_type in ['Savings','savings']:
            customer_collection[account_name].savings._withdraw_()

        '''display customer account after withdrawing'''
        
        print(customer_collection[account_name])
        
        '''creditcard charge method'''
        
    elif user_choice == '4':
        
        '''prompt user to choose customer name and proceed the method'''
        account_name,account_type = name_lookup()
        
        '''perform credit card charge action'''

        customer_collection[account_name].credit.__credit_card_charge__()

        print(customer_collection[account_name])

    ## Ivan: Credit card payment and exit HERE
    ## ivan, he said that exit means write all the change that the user made in a csv file, so I think we will use 'w' for open command. Simply perform like view customer option but display it in a new csv file. Then terminate the function. 